In [10]:
import json
from scipy.spatial import distance

In [4]:
file_path = '..\\Data\\aligned\\20170207-095506_nb-NO-aihub_aligned.json'

f = open(file_path , encoding='utf-8')
aligned_transcripts = json.load(f)

In [5]:
edit_distance = 0
for alignment in aligned_transcripts:
    if alignment[0] != alignment[1]:
        edit_distance += 1

print('WER:', round(edit_distance / len(aligned_transcripts), 3))

WER: 0.147


In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

In [15]:
# NOTE: perhaps make all words lowercase
embedded = [[model.encode(x[0]), model.encode(x[1])] for x in aligned_transcripts]
print(embedded[0])

[array([ 0.01758078, -0.0027376 , -0.0382542 , -0.04189986, -0.0536046 ,
        0.0270444 , -0.00285156, -0.04651115, -0.00723145, -0.04229123,
       -0.02418377,  0.00646215, -0.01823141,  0.01912009,  0.02715531,
        0.03859247,  0.00852604, -0.04608215, -0.00477238, -0.00212284,
        0.01424717,  0.05801569,  0.02017811,  0.01029828,  0.01391131,
        0.00421472,  0.0050398 , -0.00407265,  0.00876516, -0.0272677 ,
       -0.00505699, -0.00241932,  0.00604788, -0.02733153, -0.00203374,
       -0.03312067, -0.02645745, -0.01031437,  0.01881628, -0.05652482,
        0.01769509, -0.02128866, -0.02334602, -0.03802466,  0.00044153,
       -0.01689991, -0.00509721,  0.00618165,  0.03103199, -0.00454587,
       -0.01733579,  0.00493296, -0.04231452, -0.02654593, -0.01126566,
        0.02020068, -0.0089625 ,  0.01296299, -0.02678655,  0.03476659,
        0.04023831,  0.00445583, -0.01388247, -0.00262607, -0.02890073,
        0.02826607, -0.02522166,  0.00055787, -0.00413824,  0.0

In [20]:
sum_of_distance = 0
biggest_difference = [[], 0]


for alignment, embedding in zip(aligned_transcripts, embedded):
    cos = distance.cosine(embedding[0], embedding[1])
    sum_of_distance += cos
    if cos > biggest_difference[1]:
        biggest_difference[1] = cos
        biggest_difference[0] = alignment

print(sum_of_distance)
print(sum_of_distance / len(aligned_transcripts))
print(biggest_difference)

197.18830132484436
0.02015828065066902
[['.', 'habiliteringsfeltet'], 0.8140797317028046]
